# Optical spectra from sternheimer 
[Link to tutorial](https://www.octopus-code.org/documentation/13/tutorial/response/optical_spectra_from_sternheimer/)

We have just seen how to calculate optical spectra in the time domain with a finite perturbation, and in a frequency-domain, linear-response matrix formulation with the Casida equation. Now we will try a third approach, which is in the frequency domain and linear response but rather than using a pseudo-eigenvalue equation as in Casida, uses a self-consistent linear equation, the Sternheimer equation. This approach is also known as density-functional perturbation theory. It has superior scaling, is more efficient for dense spectra, and is more applicable to nonlinear response. One disadvantage is that one needs to proceed one frequency point at a time, rather than getting the whole spectrum at once. We will find we can obtain equivalent results with this approach for the optical spectra as for time propagation and Casida, by calculating the polarizability and taking the imaginary part.


In [ ]:
import math
import os

In [ ]:
!mkdir -p 4_Optical_spectra_from_Sternheimer

In [ ]:
cd 4_Optical_spectra_from_Sternheimer

## Ground state

Before doing linear response, we need to obtain the ground state of the system, for which we can use the same input file as for [Optical spectra from Casida](3_Optical_spectra_from_casida.ipynb), but we will use a tighter numerical tolerance, which helps the Sternheimer equation to be solved more rapidly. Unlike for Casida, no unoccupied states are required. If they are present, they won't be used anyway.


In [ ]:
%%writefile inp

stdout = 'stdout_gs.txt'
stderr = 'stderr_gs.txt'

CalculationMode = gs
UnitsOutput = eV_angstrom

Radius = 6.5*angstrom
Spacing = 0.24*angstrom

CH = 1.097*angstrom
%Coordinates
 "C" |           0 |          0 |           0
 "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
 "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
 "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
 "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
%

ConvRelDens = 1e-7

In [ ]:
!octopus

## Linear response

Change the [CalculationMode](https://www.octopus-code.org/documentation//13/variables/calculation_modes/calculationmode) to `em_resp` and add
the lines below to the input file.


In [ ]:
%%writefile inp

stdout = 'stdout_em_resp.txt'
stderr = 'stderr_em_resp.txt'

CalculationMode = em_resp
UnitsOutput = eV_angstrom

Radius = 6.5*angstrom
Spacing = 0.24*angstrom

CH = 1.097*angstrom
%Coordinates
 "C" |           0 |          0 |           0
 "H" |  CH/sqrt(3) | CH/sqrt(3) |  CH/sqrt(3)
 "H" | -CH/sqrt(3) |-CH/sqrt(3) |  CH/sqrt(3)
 "H" |  CH/sqrt(3) |-CH/sqrt(3) | -CH/sqrt(3)
 "H" | -CH/sqrt(3) | CH/sqrt(3) | -CH/sqrt(3)
%

ConvRelDens = 1e-7

%EMFreqs
5 | 0*eV | 8*eV
9 | 10*eV | 12*eV
%
EMEta = 0.1*eV

Preconditioner = no
LinearSolver = qmr_dotp
ExperimentalFeatures = yes

The frequencies of interest must be specified, and we choose them based on the what we have seen from the Casida spectrum. The block above specifies 5 frequencies spanning the range 0 to 8 eV (below the resonances) and 9 frequencies spanning the range 10 to 12 eV (where there are peaks). If we didn’t know where to look, then looking at a coarse frequency grid and then sampling more points in the region that seems to have a peak (including looking for signs of resonances in the real part of the polarizability) would be a reasonable approach. We must add a small imaginary part (ηη) to the frequency in order to be able to obtain the imaginary part of the response, and to avoid divergence at resonances. The resonances are broadened into Lorentzians with this width. The larger the ηη, the easier the SCF convergence is, but the lower the resolution of the spectrum.

To help in the numerical solution, we turn off the preconditioner (which sometimes causes trouble here), and use a linear solver that is experimental but will give convergence much faster than the default one.

The time requiered to run octopus is in the order of half an hour.

In [ ]:
!octopus

## Spectrum

This Python script can extract the needed info out of the files in the `em_resp` directory:

In [ ]:
c = 137.035999679

# List directories matching 'em_resp/freq*'
ls_output = os.listdir("em_resp")
freqs = sorted(
    [
        float(filename.split("_")[1])
        for filename in ls_output
        if filename.startswith("freq_")
    ]
)
print("# Energy (eV)    Re alpha         Im alpha    Cross-section (A^2)")
format_str = "{:8.3f}     {:13.7f}     {:13.7f}     {:13.7f}"  # Updated to include 4 decimal places

for freq in freqs:
    freq_str = f"{freq:.4f}"  # Format freq with 4 decimal places
    if freq == 0.0000:
        Im_alpha = 0
        energy = 0
        cross_section = 0
    else:
        cross_section_file = open(f"em_resp/freq_{freq_str}/cross_section")
        crossbits = cross_section_file.read().split()
        energy = float(crossbits[26])
        cross_section = float(crossbits[27])  # isotropic average
        Im_alpha = c * cross_section / (4 * math.pi * energy)
        cross_section_file.close()

    alpha_file = open(f"em_resp/freq_{freq_str}/alpha")
    alphabits = alpha_file.read().split()
    Re_alpha = float(alphabits[15])
    alpha_file.close()

    print(format_str.format(energy, Re_alpha, Im_alpha, cross_section))

[Go to *5_Triplet_excitations.ipynb*](5_Triplet_excitations.ipynb)